In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

#orhtogonal

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

σ⁺ = (σˣ + im*σʸ)/2  
σ⁻ = (σˣ - im*σʸ)/2   

tr(σ⁻* σ⁺) #not orthoganal 

1.0 + 0.0im

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & -w
   \end{pmatrix}/2
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = r \sigma^- = \begin{pmatrix} 0 & r \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

$J_2 = e \sigma^+ = \begin{pmatrix} 0 & 0 \\ e & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 &  0 \\ \sqrt{\gamma_2} & 0 
   \end{pmatrix}
$



$
J_3 = p_x \sigma^x = \begin{pmatrix} 0 & p_x \\ p_x & 0 
   \end{pmatrix} = \sqrt{\gamma_3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
J_4 =  p_y \sigma^y = \begin{pmatrix} 0 & -i p_y \\ i p_y & 0 
   \end{pmatrix} = \sqrt{\gamma_4}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
J_5 =  p_z \sigma^z = \begin{pmatrix} p_z & 0 \\ 0 & -p_z
   \end{pmatrix} = \sqrt{\gamma_5}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [ ]:
J₂ˢʸᵐᵇ  = p * [ 0    1
                1    0. ] # depolarization

J₃ˢʸᵐᵇ = p * [ 0   -im
               im   0  ] # depolarization

J₄ˢʸᵐᵇ = p * [ 1    0
               0   -1. ] # depolarization

In [18]:
# Define polynomial variables
@polyvar w r e p px py pz

Hˢʸᵐᵇ = [ w   0
          0   -w ]/2


J₁ˢʸᵐᵇ = r * σ⁻ 

#J₂ˢʸᵐᵇ  = e * σ⁺ 

J₃ˢʸᵐᵇ = p * σˣ 

J₄ˢʸᵐᵇ = p * σʸ

J₅ˢʸᵐᵇ = p * σᶻ

#J₆ˢʸᵐᵇ = px * σˣ 

#J₇ˢʸᵐᵇ = py * σʸ

#J₈ˢʸᵐᵇ = pz * σᶻ # pure de_phasing



Jˢʸᵐᵇ = [J₁ˢʸᵐᵇ, J₃ˢʸᵐᵇ, J₄ˢʸᵐᵇ, J₅ˢʸᵐᵇ] #, J₈ˢʸᵐᵇ] # J₂ˢʸᵐᵇ   J₆ˢʸᵐᵇ, J₇ˢʸᵐᵇ,


function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function g_objective(γᵢ, Hˢʸᵐᵇ, Jˢʸᵐᵇ, file_name, states)

    objₑₓ = 0

    for state in states # loop over initial states

        tᵗʳⁿ, ρᵗʳⁿ = read_timeevolution(file_name, state, γᵢ)
        
        #if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ) #[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ)#[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, Jˢʸᵐᵇ )
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective (generic function with 1 method)

In [4]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 1000

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [5]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_files = vcat(train_states, test_states);

In [22]:
include("../TSSOS_KOSSAKOWSKI//CtrlSID.jl")

Main.CtrlSID

In [27]:
using Statistics

In [28]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

op_num = 4

tests_data_file_name = "POP_NEW_LME_AMZ_ORT_op"*string(op_num)*"_rand_trn4_tst10_"*date_and_time_string * ".h5"


FminGammas = []
FmedianGammas = []
FmeanGammas = []
Omegas = []

for γᵢ in γ

    F = []

    println("γ =  "*γᵢ)

    poly = g_objective(γᵢ, Hˢʸᵐᵇ, Jˢʸᵐᵇ[1:op_num], evol_data_file_name, train_states)
   
    sol = local_rand_min(poly)

    #sol, opt_val, status = CtrlSID.jump(poly)
    #sol, opt_val, status = CtrlSID.tssos(polyGEXY)


    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    Jˢⁱᵈ = [DenseOperator(basis, j) for j in subs.(Jˢʸᵐᵇ[1:op_num], sol)]
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   

        γ_group["omega"] = convert(Float64, omega)

        if op_num >= 1 
            relaxation = subs(r^2, sol)
            γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        end
        #if op_num >= 2 
        #    exitation = subs(e^2, sol)
        #    γ_group["gamma_exitation"] = convert(Float64, exitation) 
        #end
        #if op_num >= 3 
        #    gx = subs(px^2, sol)
        #    γ_group["gamma_x"] = convert(Float64,  gx) 
        #end
        #if op_num >= 4 
        #    gy = subs(py^2, sol)
        #    γ_group["gamma_y"] = convert(Float64,  gy) 
        #end
        #if op_num >= 5
        #    gz = subs(pz^2, sol)
        #    γ_group["gamma_z"] = convert(Float64,  gz) 
        #end
        
        #γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)

        for o in 1:op_num
            γ_group["J"*string(o)] = convert.(ComplexF64, subs.(Jˢʸᵐᵇ[o], sol))
        end

    end

    #println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), Jˢⁱᵈ)
        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        end
        
        FminState = minimum(Fᴸᴹᴱₑₓ)
        FmedianState = mean(Fᴸᴹᴱₑₓ)
        FmeanState = mean(Fᴸᴹᴱₑₓ)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)

        #println(minimum(FminState))

        # Calculate the mean
        F_mean_value = mean(FmeanStates)

        # Calculate the median
        F_median_value = median(FmedianStates)

        # Calculate the min
        F_min_value = minimum(FminStates)

        push!(FminGammas, F_min_value)
        push!(FmedianGammas, F_median_value)
        push!(FmeanGammas, F_mean_value)

    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)


end

println(tests_data_file_name)

γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9996374557106616
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9999988473237456
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9999934560301337
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9999536091009134
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9995531421441672
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.992008949533805
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9809199761520405
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.9463783112557727
POP_NEW_LME_AMZ_ORT_op4_rand_trn4_tst10_2024-Jul-07_at_10-09.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9990441127158606
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9998695404236922
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9995657636515676
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9984769635902724
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9948427236554982
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.962100741490147
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9341345447520071
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.8990704961192588
POP_NEW_LME_AMZ_ORT_op4_rand_trn4_tst10_2024-Jul-07_at_09-19.h5


In [24]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

op_num = 4

tests_data_file_name = "POP_NEW_LME_AMZ_ORT_op"*string(op_num)*"_tssos_trn4_tst10_"*date_and_time_string * ".h5"

for γᵢ in γ

    F = []

    println("γ =  "*γᵢ)

    poly = g_objective(γᵢ, Hˢʸᵐᵇ, Jˢʸᵐᵇ[1:op_num], evol_data_file_name, train_states)
   
    #sol = local_rand_min(poly)

    #sol, opt_val, status = CtrlSID.jump(poly)
    sol, opt_val, status = CtrlSID.tssos(poly)

    println(status)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    Jˢⁱᵈ = [DenseOperator(basis, j) for j in subs.(Jˢʸᵐᵇ[1:op_num], sol)]
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   

        γ_group["omega"] = convert(Float64, omega)

        if op_num >= 1 
            relaxation = subs(r^2, sol)
            γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        end
        #if op_num >= 2 
        #    exitation = subs(e^2, sol)
        #    γ_group["gamma_exitation"] = convert(Float64, exitation) 
        #end
        #if op_num >= 3 
        #    gx = subs(px^2, sol)
        #    γ_group["gamma_x"] = convert(Float64,  gx) 
        #end
        #if op_num >= 4 
        #    gy = subs(py^2, sol)
        #    γ_group["gamma_y"] = convert(Float64,  gy) 
        #end
        #if op_num >= 5
        #    gz = subs(pz^2, sol)
        #    γ_group["gamma_z"] = convert(Float64,  gz) 
        #end
        
        #γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)

        for o in 1:op_num
            γ_group["J"*string(o)] = convert.(ComplexF64, subs.(Jˢʸᵐᵇ[o], sol))
        end

    end

    #println()

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), Jˢⁱᵈ)
        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        end
        
        FminState = minimum(Fᴸᴹᴱₑₓ)
        push!(F, FminState)

        #println(minimum(FminState))
    
    end
    
    println("Minimal fidelity for "*γᵢ*": ", minimum(F))

end

println(tests_data_file_name)

γ =  0.079477
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.46502625570785155
γ =  0.25133
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.24336351654579585
γ =  0.79477
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.298941325597749
γ =  2.5133
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.4370434245130651
γ =  7.9477
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.6797693370216691
γ =  25.133
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.4797981588191619
γ =  79.477
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.6612431243934295
γ =  251.33
LocalD1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.8990705070720013
POP_NEW_LME_AMZ_ORT_op4_tssos_trn4_tst10_2024-Jul-07_at_09-11.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.46502625473571035
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.24336351538998607
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.2989413162915427
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.4370432490603813
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.6797673148750556
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.4797893426424587
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.6613437767951185
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.6987666435222311
POP_NEW_LME_AMZ_ORT_op4_tssos_trn4_tst10_2024-Jul-07_at_09-05.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9990441158212405
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9998695404260469
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9995657636506401
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9984769635380284
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9948427236628077
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.962100741558644
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9341345446974678
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.8990704963077147
POP_NEW_LME_AMZ_ORT_op4_random_trn4_tst10_2024-Jul-07_at_08-49.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9990441118037822
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9998695404479757
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9995657636522979
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9984769636263768
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.994986434720013
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9621007415585846
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.934134544697641
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.8990704961816758
POP_NEW_LME_AMZ_ORT_ph_op5_random_trn4_tst10_2024-Jul-07_at_08-34.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9872526308346593
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.996983167519077
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9986247466518567
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9986411756971114
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9959249762941988
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9817315083026658
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9848382996195774
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.9893029519926322
POP_NEW_LME_AMZ_ORT5_op7_random_trn4_tst10_2024-Jul-07_at_08-21.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9872526476792106
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9969831658787631
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9986247473005518
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9986411757036349
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9959249763127018
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9817315082996174
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9848382996177131
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.9893029519748909
POP_NEW_LME_AMZ_ORT5_op8_random_trn4_tst10_2024-Jul-07_at_08-00.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9990441130879371
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9998686622721551
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9995561697300122
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9983702110453525
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9948427237035865
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9674328202238615
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9347070553092605
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.9231793653739864
POP_NEW_LME_AMZ_ORT5_op5_random_trn4_tst10_2024-Jul-07_at_07-42.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.9872526299633188
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9969831663487638
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.998624746438889
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9986411758123308
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9959249764044789
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9817315082829492
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.984838299623932
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.9893029519982499
POP_NEW_LME_AMZ_ORT5_op5_random_trn4_tst10_2024-Jul-07_at_06-54.h5


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.079477: 0.983025137251673
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.25133: 0.9924065949367566
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 0.79477: 0.9958247024167625
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 2.5133: 0.9984275339087479
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 7.9477: 0.9980828546070692
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 25.133: 0.9826918818142714
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 79.477: 0.9948038197116301
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Minimal fidelity for 251.33: 0.9954961894945644
POP_NEW_LME_AMZ2_op5_random_trn4_tst10_2024-Mar-13_at_13-49.h5
